In [1]:
import pandas as pd
import numpy as np 

#for data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#for evaluation
from sklearn.metrics import mean_absolute_error, r2_score, classification_report,confusion_matrix , accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [251]:
df = pd.read_excel('/content/data_epilepsy.xlsx',sheet_name='S-N')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 4057,Unnamed: 4058,Unnamed: 4059,Unnamed: 4060,Unnamed: 4061,Unnamed: 4062,Unnamed: 4063,Unnamed: 4064,Unnamed: 4065,Unnamed: 4066,Unnamed: 4067,Unnamed: 4068,Unnamed: 4069,Unnamed: 4070,Unnamed: 4071,Unnamed: 4072,Unnamed: 4073,Unnamed: 4074,Unnamed: 4075,Unnamed: 4076,Unnamed: 4077,Unnamed: 4078,Unnamed: 4079,Unnamed: 4080,Unnamed: 4081,Unnamed: 4082,Unnamed: 4083,Unnamed: 4084,Unnamed: 4085,Unnamed: 4086,Unnamed: 4087,Unnamed: 4088,Unnamed: 4089,Unnamed: 4090,Unnamed: 4091,Unnamed: 4092,Unnamed: 4093,Unnamed: 4094,Unnamed: 4095,Unnamed: 4096
0,100,124,153,185,210,220,216,222,240,265,298,330,362,381,391,390,390,392,395,397,397,393,380,356,334,322,320,322,338,332,317,263,195,142,108,97,80,80,85,39,...,516,592,650,667,637,574,505,450,413,399,414,448,486,449,291,-46,-457,-878,-1178,-1324,-1310,-1105,-770,-345,42,315,465,513,478,378,250,137,57,17,0,7,26,57,95,462
1,340,353,400,470,538,590,611,629,649,693,759,837,812,684,389,-231,-1250,-1768,-1816,-1637,-984,-476,-224,-14,165,314,408,454,467,437,412,379,376,339,302,248,200,174,144,152,...,304,254,82,-174,-467,-537,-450,-178,64,274,464,496,571,467,299,130,28,82,270,597,886,1071,1166,1146,1000,773,423,118,-75,-18,205,556,824,919,916,829,722,512,130,196
2,-310,93,494,789,798,552,202,-95,-262,-327,-420,-483,-508,-502,-459,-376,-266,-113,18,98,135,154,163,174,186,196,208,216,218,208,200,197,197,201,211,224,239,246,252,261,...,-21,-12,29,75,111,145,168,169,150,113,106,137,204,291,388,477,546,574,546,467,360,255,164,123,126,184,312,410,470,496,437,345,108,-216,-525,-735,-847,-721,-491,910
3,84,75,21,-68,-138,-184,-197,-186,-155,-111,-58,-26,-18,-18,-40,-56,-35,35,149,253,298,282,232,161,96,71,64,70,35,-45,-116,-169,-192,-196,-173,-131,-82,-38,-8,-4,...,-91,-191,-246,-261,-248,-228,-197,-172,-150,-133,-123,-113,-109,-93,-71,-54,-42,-32,-17,29,110,200,271,285,236,142,50,-15,-61,-83,-119,-148,-171,-195,-218,-234,-238,-209,-165,-212
4,343,311,284,274,260,237,165,-33,-271,-425,-418,-254,-104,-14,16,24,23,11,4,20,40,67,99,127,130,126,133,124,108,54,-5,-45,-61,-65,-52,-46,-25,-15,-4,-12,...,2,-70,-118,-155,-201,-283,-368,-363,-333,-271,-203,-126,-31,48,86,102,124,125,132,144,187,263,343,411,433,417,397,395,414,435,439,428,423,430,472,515,527,480,397,217


In [3]:
target = pd.read_excel('/content/data_epilepsy.xlsx',sheet_name='target S-Z')

In [252]:
x=df.iloc[:,0:]
y=target

In [253]:
from sklearn.model_selection import train_test_split
x_Train, x_Test, y_Train, y_Test =train_test_split(x,y,test_size =0.30)

In [254]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [255]:
scaled_data = scaler.transform(df)

In [256]:
from sklearn.decomposition import PCA

In [453]:
pca = PCA(n_components=3)

In [454]:
pca.fit(scaled_data)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [455]:
x_pca = pca.transform(scaled_data)

In [456]:
scaled_data.shape

(200, 4097)

In [457]:
x_pca.shape

(200, 3)

In [458]:
from sklearn.model_selection import train_test_split
x_Train_pca,x_Test_pca,y_Train,y_Test =train_test_split(x_pca,y,train_size =.8)

#DecisionTreeClassifier

In [459]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train = pca.fit_transform(x_Train)
X_test = pca.transform(x_Test)

In [460]:
# Necessary imports
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# Creating the hyperparameter grid 
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 7),
              "min_samples_leaf": randint(1, 7),
              "criterion": ["gini", "entropy"]}

# Instantiating Decision Tree classifier
tree = DecisionTreeClassifier()


# Instantiating RandomizedSearchCV object
tree_cv = RandomizedSearchCV(tree, param_dist, cv = 5)

tree_cv.fit(x_Train_pca, y_Train)
  
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Tuned Decision Tree Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 1, 'min_samples_leaf': 6}
Best score is 0.8625


In [461]:
y_pred = tree_cv.predict(x_Test_pca)
y_pred

array([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [462]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_Test,y_pred))

0.85


In [463]:
dtc_acc = accuracy_score(y_Test,y_pred)
print(dtc_acc)

0.85


In [464]:
results = pd.DataFrame()
results

""


In [465]:
tempResults = pd.DataFrame({'Algorithm':['Decision tree Classifier Method'], 'Accuracy':[dtc_acc]})
results = pd.concat( [results, tempResults] )
results = results[['Algorithm','Accuracy']]
results

,Algorithm,Accuracy
0,Decision tree Classifier Method,0.85


##RandomForest

In [466]:
 from sklearn.ensemble import RandomForestClassifier
 from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(x_Train_pca, y_Train)
RandomForestClassifier(...)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=Ellipsis,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [467]:
RandomForestClassifier(max_depth=3, random_state=0)
print(clf.feature_importances_)

[0.32021113 0.37411045 0.30567842]


In [468]:
y_pred = clf.predict(x_Test_pca)
y_pred

array([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [469]:
accuracy_score(y_Test, y_pred)

0.925

In [470]:
rfc_acc = accuracy_score(y_Test,y_pred)
print(rfc_acc)

0.925


In [471]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_Test, y_pred)
cm

array([[18,  2],
       [ 1, 19]])

In [472]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_Test,y_pred))

0.925


In [473]:

rfc_acc = accuracy_score(y_Test,y_pred)
print(rfc_acc)

0.925


In [474]:
tempResults = pd.DataFrame({'Algorithm':['Random Forest Classifier Method'], 'Accuracy':[rfc_acc]})
results = pd.concat( [results, tempResults] )
results = results[['Algorithm','Accuracy']]
results

,Algorithm,Accuracy
0,Decision tree Classifier Method,0.850
0,Random Forest Classifier Method,0.925


#XGBoost

In [475]:
import numpy as np
from sklearn.datasets import load_svmlight_files
from sklearn.metrics import accuracy_score

from xgboost.sklearn import XGBClassifier

In [476]:
model =XGBClassifier()
eval_set =[(x_Test_pca,y_Test)]
model.fit(x_Train_pca,y_Train,early_stopping_rounds= 10, eval_metric ='logloss',eval_set=eval_set,verbose=True)

[0]	validation_0-logloss:0.632165
Will train until validation_0-logloss hasn't improved in 10 rounds.
[1]	validation_0-logloss:0.581129
[2]	validation_0-logloss:0.537903
[3]	validation_0-logloss:0.500946
[4]	validation_0-logloss:0.471037
[5]	validation_0-logloss:0.445122
[6]	validation_0-logloss:0.424104
[7]	validation_0-logloss:0.402066
[8]	validation_0-logloss:0.382978
[9]	validation_0-logloss:0.366404
[10]	validation_0-logloss:0.353653
[11]	validation_0-logloss:0.340427
[12]	validation_0-logloss:0.33064
[13]	validation_0-logloss:0.320005
[14]	validation_0-logloss:0.307552
[15]	validation_0-logloss:0.300319
[16]	validation_0-logloss:0.292465
[17]	validation_0-logloss:0.286982
[18]	validation_0-logloss:0.28045
[19]	validation_0-logloss:0.271715
[20]	validation_0-logloss:0.263995
[21]	validation_0-logloss:0.256026
[22]	validation_0-logloss:0.24853
[23]	validation_0-logloss:0.245558
[24]	validation_0-logloss:0.240003
[25]	validation_0-logloss:0.236649
[26]	validation_0-logloss:0.232114


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [477]:
#make predictions for test data
predictions = model.predict(x_Test_pca)


In [478]:
y_xgb_pred =model.predict(x_Test_pca)
print(y_Test)
print(y_xgb_pred)

cm = confusion_matrix(y_xgb_pred,y_Test)
print(cm)

     target
9         1
143       0
26        1
167       0
21        1
132       0
163       0
5         1
197       0
14        1
15        1
179       0
139       0
54        1
56        1
10        1
96        1
63        1
113       0
123       0
128       0
121       0
177       0
186       0
83        1
72        1
64        1
0         1
175       0
122       0
8         1
60        1
18        1
27        1
116       0
51        1
170       0
130       0
107       0
146       0
[1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0
 0 0 1]
[[16  0]
 [ 4 20]]


In [479]:
# evoluate predictions
accuracy = accuracy_score(y_Test, predictions)
print("Accuracy: %.2f%%" %(accuracy * 100.0))

Accuracy: 90.00%


In [480]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_Test,y_pred))

0.925


In [481]:
xgb_acc = accuracy_score(y_Test, y_pred)
print(xgb_acc)

0.925


In [482]:
tempResults = pd.DataFrame({'Algorithm':['XGBoost Classifier Method'], 'Accuracy':[xgb_acc]})
results = pd.concat( [results, tempResults] )
results = results[['Algorithm','Accuracy']]
results

,Algorithm,Accuracy
0,Decision tree Classifier Method,0.850
0,Random Forest Classifier Method,0.925
0,XGBoost Classifier Method,0.925
